In [1]:
number_of_shots = 3

In [2]:
experiment_name = "2024.05.08-01_Mixtral_7B_instruct_API_3-few-shots_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [9]:
# top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data_but_with_different_classes.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [10]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx")

In [11]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [12]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [13]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

# system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are {number_of_shots} examples of expected classification given an input text:

# """+few_shots+"""

# Answer with the label only using the format:---LABEL---
# """


In [14]:
data_sel = data #.iloc[:5]

In [15]:
# Sposta il prompt dentro il ciclo

In [16]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]

    few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]    
    
    few_shots = ""

    for train_data_id in few_shots_list:

        # print("##################################################")
        # print(train_data_id)

        train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

        # print(train_row)

        # print(train_row.text.values[0])
        # print(train_row.sdg.values[0])

        input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
        output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

        few_shots += input_text
        few_shots += output_text    
    
    system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

    The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

    The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

    If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

    The following are {number_of_shots} examples of expected classification given an input text:

    """+few_shots+"""

    Answer with the label only using the format:---LABEL---
    """    
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": model_name,
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name.replace("/","_")}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 11---
1 6099 0  ---SDG 0---

The text is about a model's photo shoot for the Sports Illustrated Swimsuit issue in the Maldives. It does not contain any information related to the Sustainable Development Goals.
2 6471 0  ---SDG 16---

The text discusses the legal case of Afroz Khan Pathan, who was convicted in the 2006 Aurangabad arms haul case in India. The Bombay High Court has suspended his life sentence and granted him conditional bail. The court also examined the claim made by two co-accused that Pathan had visited Bangladesh to collect funds for terror activities in India, but found no prima facie evidence to support this claim. The text highlights the importance of the rule of law and fair trial, which are part of SDG 16 (Peace, Justice and Strong Institutions).
3 1786 0  ---SDG 9--- (Industry, Innovation and Infrastructure) as the text is about the discovery and naming of minor planets, which involves innovation and scientific advancement.
4 2143 0  ---SDG 7---


47 1427 0  ---SDG 15--- (Life on Land) The article focuses on species diversity, which is a key aspect of Life on Land (SDG 15).
48 2023 0  ---SDG 16---
49 854 0  ---SDG 0---
50 2541 0  ---SDG 3--- (This text discusses the negative health effects of certain breakfast foods, which aligns with SDG 3: Good Health and Well-being.)
51 6224 0  ---SDG 0---
52 6243 0  ---SDG 16---

(The classification is made under SDG 16, which is "Peace, Justice and Strong Institutions", as the text discusses the Uniform Civil Code, a law that aims to promote justice and equality in the country.)
53 4104 0  ---SDG 0---

The text is about the health update of actor Jamie Foxx and does not relate to any specific SDG.
54 1497 0  ---SDG 9---
55 2055 0  ---SDG 3--- (This text refers to the discovery of new cell types and hidden molecular features, which can be applied to medical and health-related fields, which falls under SDG 3: Good Health and Well-being)
56 2463 0  ---SDG 0---

The text is about personnel chan

85 1086 1  ---SDG 1---

The text discusses the cash benefits and social package entitlements provided to certain groups, such as disabled WWII veterans and survivors. The increase in these payments has contributed to the eradication of poverty among pensioners in absolute terms. This aligns with SDG 1, which aims to end poverty in all its forms everywhere.
86 330 1  "---SDG 1---"
87 375 1  ---SDG 1---
88 2745 1  ---SDG 1---

The text discusses the expansion of the Canadian Child Benefit (CCB), a benefit program targeted towards low-income families, but also aiming to support middle-class families. The proposed accelerated indexation of the CCB will provide additional financial support to these families over a five-year period. This measure can help alleviate poverty and contribute to the goal of no poverty (SDG 1).
89 6329 1  ---SDG 1---
90 5668 1  ---SDG 10---

The text discusses the relationship between income inequality and economic growth, which relates to SDG 10: Reduced Inequalit

126 720 2  ---SDG 2---

(The text suggests addressing food security and agricultural productivity, which aligns with SDG 2.)
127 6780 2  ---SDG 2---

The text is about food security, which is directly related to the Sustainable Development Goal 2: Zero Hunger. The text discusses the impact of food insecurity on children, including their mental and physical well-being, as well as the role of parents and the government in ensuring food security. The text also mentions the importance of children's accurate reporting of their food security experiences, which is crucial in addressing food insecurity.
128 2407 2  ---SDG 12--- (Sustainable consumption and production)
129 3672 2  ---SDG 2--- (Goal 2 focuses on ending hunger, achieving food security and improved nutrition and promoting sustainable agriculture.)
130 4315 2  ---SDG 2---

The text emphasizes the importance of monitoring food insecurity among households with children, which aligns with the focus of SDG 2, which is to end hunger, ac

158 654 2  ---SDG 2---

The text discusses the impact of high and volatile food prices, which can lead to irreversible harm such as malnutrition, mortality, and withdrawal of children from education. This is closely related to SDG 2, which aims to end hunger, achieve food security and improved nutrition, and promote sustainable agriculture. The text also mentions natural factors that affect agricultural output, which is relevant to the sustainable agriculture aspect of SDG 2.
159 4178 2  ---SDG 2---
160 5812 2  ---SDG 2---
161 5130 2  ---SDG 2---
162 4078 2  ---SDG 9---
163 646 2  Based on the text provided, it seems to be discussing the issue of child food insecurity and its impact on children's well-being and development. Therefore, I would classify this text as:

---SDG 2---

This classification is based on the fact that the text discusses the negative consequences of child food insecurity, which aligns with SDG 2's focus on ending hunger, achieving food security, and improving nutr

...

870 3877 14  ---SDG 14---
871 71 14  ---SDG 14---
872 5815 14  ---SDG 13---
873 1388 14  Based on the given text, the classification under the Sustainable Development Goals (SDGs) is:

---SDG 14---

This goal is focused on life below water, which includes the sustainable use of marine resources. The text discusses the characterization of small-scale fisheries (SSF), which is a relevant topic for the "Friends of SSF" platform and is being tested through existing case studies. Additionally, a socio-economic survey is being conducted to perform a wide-scale application of the matrix at the regional level. The text also mentions modern fishing operations and advanced technologies used in certain SSF, which aligns with the focus on sustainable use of marine resources.
874 3404 14  ---SDG 14---
875 2212 14  ---SDG 14---

The text is about the whaling quota and its impact on whale stocks. SDG 14 is focused on life below water, which includes the conservation and sustainable use of oceans, sea

919 3281 15  ---SDG 15---
920 1767 15  ---SDG 11---
921 1957 15  ---SDG 13--- (Climate Action) as the text discusses the growth in global emissions and their impact on climate change.
922 980 15  ---SDG 15---
923 1299 15  ---SDG 15---
924 5980 15  ---SDG 15---
925 2985 15  ---SDG 15---
926 1515 15  ---SDG 13---
927 6479 15  ---SDG 15---
928 2296 15  ---SDG 15--- \\
This text pertains to SDG 15, which is Life on Land. The text discusses the National Forest Programme in Cambodia, which aims to allocate 2 million hectares of production forest for community forests (CF) management. The CF subdecree and guidelines for forest-dependent communities have been developed, and communities have legal rights to manage their forests for 15 years. The text also highlights the challenges faced in achieving approval for CF management plans, but it does not mention any other SDGs.
929 3610 15  ---SDG 15---
930 152 15  ---SDG 15---
931 658 15  ---SDG 15---
932 1672 15  ---SDG 15---
933 3170 15  ---SDG 15

964 20 16  ---SDG 16---
965 4754 16  ---SDG 16---
966 930 16  ---SDG 16---
967 4430 16  ---SDG 16---

The article discusses the legitimacy of judicial review of a constitution's federal and structural provisions, which is closely related to SDG 16, which aims to promote peaceful and inclusive societies for sustainable development, provide access to justice for all, and build effective, accountable, and inclusive institutions at all levels. The article's focus on the legitimacy of judicial review in the context of democracy and constitutional rights aligns with the goals of SDG 16 to promote the rule of law and access to justice.
968 2761 16  ---SDG 16---

Explanation:
The text discusses the implementation and impact of electronic government in the Sabah State public administration, which aligns with SDG 16's focus on promoting peaceful and inclusive societies for sustainable development, providing access to justice for all, and building effective, accountable, and inclusive institution

987 2092 16  ---SDG 5---
988 2472 16  ---SDG 12--- (Responsible Consumption and Production) as the text discusses strategies for promoting responsible investment and environmental management.
989 3167 16  ---SDG 16---

The text discusses the development and validation of the Support for Diplomacy Scale (SDS), which measures the unidimensional nature of peacemaking and militaristic attitudes. The SDS is associated with various political and social attitudes, such as authoritarianism, dogmatism, religious fundamentalism, and immigration attitudes. The SDS also mediates the relation between just peacemaking and various measures of militarism and resistance to moral disengagement from war. These findings are relevant to SDG 16, which promotes peaceful and inclusive societies, access to justice, and effective, accountable, and inclusive institutions.
990 5026 16  ---SDG 16---
(This classification is given because the text discusses the expansion of personal jurisdiction over transnational c

1011 2370 16  ---SDG 11---

The article focuses on exploring the concept of achieving the "right to the city" for marginalized communities and developing a toolkit of indicators for urban regeneration using human rights instruments and best practices. This contributes to the literature on realizing economic, social, and cultural rights encompassed in the "right to the city." The interdisciplinary approach involves human rights law, urban planning, housing studies, community development, housing law, and social policy. The primary qualitative research conducted by the author supports the findings. The main goal is to meet human rights through urban regeneration, which aligns with SDG 11: Sustainable Cities and Communities.
1012 2870 16  ---SDG 16--- //goal 16 is about promoting peaceful and inclusive societies for sustainable development, providing access to justice for all and building effective, accountable and inclusive institutions at all levels. The text provided does not seem to f

In [17]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [18]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    #print(res)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
        sdg_pred = res[3]['choices'][0]['message']['content']
    except:
        sdg_pred = res
    res4df = (res[0], res[1], res[2], sdg_pred)    
    print(res4df)
    tuple_list.append(res4df)

('1015', 4695, 16, ' ---SDG 16---')
('582', 714, 9, ' ---SDG 9---')
('2', 6471, 0, ' ---SDG 16---\n\nThe text discusses the legal case of Afroz Khan Pathan, who was convicted in the 2006 Aurangabad arms haul case in India. The Bombay High Court has suspended his life sentence and granted him conditional bail. The court also examined the claim made by two co-accused that Pathan had visited Bangladesh to collect funds for terror activities in India, but found no prima facie evidence to support this claim. The text highlights the importance of the rule of law and fair trial, which are part of SDG 16 (Peace, Justice and Strong Institutions).')
('782', 2180, 13, ' ---SDG 13---')
('601', 4947, 10, ' ---SDG 10---\n\nThe text discusses the relationship between income inequality and economic growth, particularly in the context of China as a transition economy. This is closely related to SDG 10, which aims to reduce inequality within and among countries, including efforts to reduce income inequa

In [19]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [20]:
results_processed

,cnt,id,true_label,predicted_label
0,1015,4695,16,---SDG 16---
1,582,714,9,---SDG 9---
2,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...
3,782,2180,13,---SDG 13---
4,601,4947,10,---SDG 10---\n\nThe text discusses the relati...
...,...,...,...,...
1015,152,5700,2,---SDG 2---\n\nThe text emphasizes the import...
1016,276,1547,4,---SDG 4---\n\nThe text discusses the 2014 Fo...
1017,554,5840,9,---SDG 9---
1018,927,6479,15,---SDG 15---


In [21]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0---\n\nThe text is about a model's ph...
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,"---SDG 9--- (Industry, Innovation and Infrast..."
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---\n\nExplanation:\nThe text given ...
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---\n\nThe text discusses the accoun...
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---\n\nThe text discusses the due pr...


In [22]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0---\n\nThe text is about a model's ph...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,"---SDG 9--- (Industry, Innovation and Infrast...",9
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---,16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---\n\nExplanation:\nThe text given ...,16
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---\n\nThe text discusses the accoun...,16
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---\n\nThe text discusses the due pr...,16


In [23]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
0,33,33,1,33,33,1,1,1,33,33,33,33,33
1,75,75,74,75,75,74,74,74,75,75,75,75,75
10,35,35,35,35,35,35,35,35,35,35,35,35,35
11,78,78,75,78,78,75,75,75,78,78,78,78,78
12,60,60,60,60,60,60,60,60,60,60,60,60,60
13,65,65,64,65,65,64,64,64,65,65,65,65,65
14,56,56,56,56,56,56,56,56,56,56,56,56,56
15,59,59,57,59,59,57,57,57,59,59,59,59,59
16,56,56,52,56,56,52,52,52,56,56,56,56,56


In [24]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [25]:
results_processed_merged = results_processed_merged[~results_processed_merged.predicted_label_clean.isna()]

In [26]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.97      0.53      0.69        60
           1       0.71      0.88      0.79        60
           2       0.92      0.77      0.84        60
           3       0.81      0.93      0.87        60
           4       0.87      0.92      0.89        60
           5       0.85      0.88      0.87        60
           6       1.00      0.88      0.94        60
           7       0.94      0.83      0.88        60
           8       0.58      0.75      0.66        60
           9       0.67      0.80      0.73        60
          10       0.86      0.50      0.63        60
          11       0.74      0.97      0.84        60
          12       0.90      0.90      0.90        60
          13       0.78      0.85      0.82        60
          14       0.96      0.90      0.93        60
          15       0.90      0.88      0.89        60
          16       0.91      0.85      0.88        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# AGGIUNTO IL 2024.05.07:

In [28]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [29]:
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0---\n\nThe text is about a model's ph...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...,16


In [30]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.86      0.53      0.66        60
           1       0.71      0.88      0.79        60
           2       0.92      0.77      0.84        60
           3       0.81      0.93      0.87        60
           4       0.87      0.92      0.89        60
           5       0.85      0.88      0.87        60
           6       1.00      0.88      0.94        60
           7       0.94      0.83      0.88        60
           8       0.58      0.75      0.66        60
           9       0.67      0.80      0.73        60
          10       0.86      0.50      0.63        60
          11       0.74      0.97      0.84        60
          12       0.90      0.90      0.90        60
          13       0.78      0.85      0.82        60
          14       0.96      0.90      0.93        60
          15       0.90      0.88      0.89        60
          16       0.91      0.85      0.88        60

    accuracy    

In [31]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [32]:
report_df

,precision,recall,f1-score,support
0,86.486486,53.333333,65.979381,60.00000
1,70.666667,88.333333,78.518519,60.00000
2,92.000000,76.666667,83.636364,60.00000
3,81.159420,93.333333,86.821705,60.00000
4,87.301587,91.666667,89.430894,60.00000
5,85.483871,88.333333,86.885246,60.00000
6,100.000000,88.333333,93.805310,60.00000
7,94.339623,83.333333,88.495575,60.00000
8,58.441558,75.000000,65.693431,60.00000
9,66.666667,80.000000,72.727273,60.00000


In [33]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [34]:
print(precision, recall, accuracy, f1_score)

84.02418743949687 82.54901960784314 82.54901960784314 82.4070397845374


In [35]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

84.0 82.5 82.5 82.4
